In [10]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from dbcon import get_session, init_db, Tabledata



In [11]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
#API_HOST = os.getenv("API_HOST")
#season = 2024
#league_id = 39

In [12]:
url = "https://api.football-data.org/v4/competitions/PD/standings/"
headers = {
	"x-Auth-Token": API_KEY,
}

In [13]:
response = requests.get(url, headers=headers)
result = response.json()['standings'][0]['table']

In [14]:
rows = []
columns = ["position", "team_name", "matches", "points", "score_diff", "goals"]

In [15]:
for team in result:
    position = int(team['position'])
    team_name = str(team['team']['name'])
    matches = int(team['playedGames'])
    points = int(team['points'])
    score_diff = int(team['goalDifference'])
    goals = int(team['goalsFor'])
    rows.append([position, team_name, matches, points, score_diff, goals])

#create a datafram with the extracted data
df = pd.DataFrame(rows, columns=columns)
print(df)

    position                  team_name  matches  points  score_diff  goals
0          1             Real Madrid CF        3       9           5      6
1          2              Athletic Club        3       9           3      6
2          3              Villarreal CF        3       7           7      8
3          4               FC Barcelona        3       7           4      7
4          5  RCD Espanyol de Barcelona        3       7           2      5
5          6                  Getafe CF        3       6           0      4
6          7                   Elche CF        3       5           2      4
7          8        Real Betis Balompié        4       5           0      4
8          9                Valencia CF        3       4           2      4
9         10   Rayo Vallecano de Madrid        3       4           1      4
10        11           Deportivo Alavés        3       4           0      3
11        12                 Sevilla FC        3       3           0      5
12        13

In [16]:
# get a csv file generated.
df.to_csv('standings.csv', index=False)

In [17]:
#inintialize db connection
session = get_session()
init_db()
session

2025-09-03 12:06:21,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-03 12:06:21,641 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-09-03 12:06:21,647 INFO sqlalchemy.engine.Engine [cached since 1032s ago] {'table_name': 'tabledata', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-09-03 12:06:21,663 INFO sqlalchemy.engine.Engine 
CREATE TABLE tabledata (
	id SERIAL NOT NULL, 
	position INTEGER NOT NULL, 
	"Team_name" VARCHAR NOT NULL, 
	"Matches" INTEGER NOT NULL, 
	points INTEGER NOT NULL, 
	score_di

<generator object get_session at 0x000002609C0B02E0>

In [ ]:
#store data in the database
db_session = next(get_session())
for index, row in df.iterrows():
    table_data = Tabledata(
        position = (row['position']),
        team_name = (row['team_name']),
        matches = (row['matches']),
        points = (row['point']),
        score_diff = (row['score_diff']),
        goals = (row['goals'])
    )
    db_session.add(table_data)
db_session.commit()

✅ Database session created successfully.


KeyError: 'point'